# Relatórios

Este notebook é relativo a geração dos relatórios em csv.

## Configuração e conexão com o Banco de Dados

In [1]:
from mysql.connector import connection
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
config = {
  'user': 'admin',
  'password': 'admin',
  'host': 'localhost',
  'database': 'melhorenvio',
  'raise_on_warnings': True
}

In [3]:
cnx = connection.MySQLConnection(**config)

## Relatório 1

Requisições por consumidor

In [4]:
query = """
SELECT authenticated_entity.id, COUNT(log.request_id)
FROM log JOIN authenticated_entity ON log.authenticated_entity_id  = authenticated_entity.id
GROUP BY authenticated_entity.id;
"""

In [5]:
df = pd.read_sql(query, cnx)
df

,id,COUNT(log.request_id)
0,00032b9b-2ed2-354c-acc2-1abe1b205879,15
1,0003e5b9-9ff3-3e5d-8386-de78ecf9f13d,6
2,00058fa2-9b7c-37a7-8bf5-4ab8d97ef1cd,11
3,000d78d6-ec09-3f39-86ee-fe6e326b9d04,7
4,00277480-ee66-35d8-9510-1d877ea62b62,7
...,...,...
9994,fff061e2-0812-3034-9f65-b48dcc9ea037,8
9995,fff42b2d-4b2c-399e-bdb5-6901850ec362,6
9996,fff452f4-aa19-362b-ac7a-e3c1e71703c8,8
9997,fff8a78f-838b-3283-9b1d-b6a3f3bcb150,13


In [6]:
df.to_csv('relatorios/requisicoes_por_consumidor.csv', index=False)

## Relatório 2

Requisições por serviço

In [7]:
query = """
SELECT service.id, COUNT(log.service_id)
FROM log JOIN service ON log.service_id  = service.id
GROUP BY service.id;
"""

In [8]:
df = pd.read_sql(query, cnx)
df

,id,COUNT(log.service_id)
0,22f8e3a6-01f7-3264-b4b5-9d178df11d06,20054
1,a5bf08bd-c030-30d5-8009-83a8c30103bf,19963
2,c3e86413-648a-3552-90c3-b13491ee07d6,19943
3,d035ffcf-914a-3007-b028-ae18f04d75b4,20148
4,eb1ce287-5797-3e45-b9a6-e6d51691257e,19892


In [9]:
df.to_csv('relatorios/requisicoes_por_servico.csv', index=False)

## Relatório 3

Tempo médio de request, proxy e kong por serviço.

In [10]:
query = """
SELECT log.service_id,
       AVG(latencies.proxy),
       AVG(latencies.kong),
       AVG(latencies.request)
FROM log JOIN latencies ON log.latencies_id  = latencies.id
GROUP BY log.service_id;
"""

In [11]:
df = pd.read_sql(query, cnx)
df

,service_id,AVG(latencies.proxy),AVG(latencies.kong),AVG(latencies.request)
0,22f8e3a6-01f7-3264-b4b5-9d178df11d06,1401.4732,12.5724,1750.1236
1,a5bf08bd-c030-30d5-8009-83a8c30103bf,1394.6939,12.4260,1750.2643
2,c3e86413-648a-3552-90c3-b13491ee07d6,1400.4764,12.5028,1747.7644
3,d035ffcf-914a-3007-b028-ae18f04d75b4,1403.1892,12.5060,1742.2690
4,eb1ce287-5797-3e45-b9a6-e6d51691257e,1401.5103,12.5024,1750.3413


In [12]:
df.to_csv('relatorios/tempos_medios_por_servico.csv', index=False)

In [13]:
cnx.close()